<a href="https://colab.research.google.com/github/heliocsantos/Projeto-da-Imers-o-Alura-Google/blob/main/Projeto_Imers%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q -U google-generativeai

# 1. Importar as ferramentas (bibliotecas) que vamos usar
import numpy as np # Ferramenta para trabalhar com números
import pandas as pd # Ferramenta para organizar dados em tabelas
import google.generativeai as genai # Ferramenta de Inteligência Artificial do Google

# 2. Coloque aqui a sua chave de API do Google
# Você precisa criar uma conta no Google Cloud Platform e ativar a API do Generative AI
from google.colab import userdata
api_key = userdata.get("SECRET_KEY")

genai.configure(api_key=api_key)

# 3. Criar o banco de dados de recursos
# Imagine que esta é uma tabela, com cada linha representando um recurso
dados_recursos = {
    "nome": ["Abrigo Acolher", "Restaurante Esperança", "Hospital Santa Casa", "Minha Residencia"],
    "endereço": ["Rua A, 123, Porto Alegre", "Rua B, 456, Porto Alegre", "Rua C, 789, Porto Alegre", "Rua das Lobélias, 380, São Paulo"],
    "tipo": ["abrigo", "refeição", "saúde", "casa"],
    "horario": ["Aberto 24h", "Almoço: 12h-14h", "Atendimento 24h", "Atendimento 24h"],
    "telefone": ["(51) 1234-5678", "(51) 9876-5432", "(51) 3456-7890", "(11) 99185-7890"]
}

# Transformamos os dados em um DataFrame (tabela organizada)
recursos = pd.DataFrame(dados_recursos)

# 4. Função para transformar texto em números (embeddings)
# A IA precisa transformar as palavras em números para entender o significado
model = "models/embedding-001" # Modelo de embedding do Google
def gerar_embeddings(texto):
  return genai.embed_content(model=model, content=texto, task_type="RETRIEVAL_DOCUMENT")["embedding"]

# 5. Gerar embeddings para cada recurso no banco de dados
recursos["embeddings"] = recursos["tipo"].apply(gerar_embeddings)

# 6. Função para encontrar recursos similares à necessidade do usuário
def encontrar_recursos(necessidade, recursos, model):
  # Transforma a necessidade do usuário em números (embedding)
  embedding_da_necessidade = genai.embed_content(model=model, content=necessidade, task_type="RETRIEVAL_QUERY")["embedding"]

  # Calcula a similaridade entre a necessidade e cada recurso
  similaridades = np.dot(np.stack(recursos["embeddings"]), embedding_da_necessidade)

  # Encontra o recurso mais similar
  indice_do_recurso = np.argmax(similaridades)
  return recursos.iloc[indice_do_recurso]

# 7. Interface de texto (exemplo)
necessidade_usuario = input("Qual a sua necessidade? ") # Pergunta ao usuário
recurso_encontrado = encontrar_recursos(necessidade_usuario, recursos, model) # Busca o recurso

# 8. Mostrar o resultado para o usuário
print("Recurso encontrado:")
print("Nome:", recurso_encontrado["nome"])
print("Endereço:", recurso_encontrado["endereço"])
print("Telefone:", recurso_encontrado["telefone"])

Qual a sua necessidade? casa
Recurso encontrado:
Nome: Minha Residencia
Endereço: Rua das Lobélias, 380, São Paulo
Telefone: (11) 99185-7890
